### OTFS Modulation Scheme

The Orthogonal Frequency Time Space (OTFS) is a modulation scheme used to improve and tackle the problems which traditional OFDM systems cannot. This is done by shifting the analysis and synthesis operations to the delay-doppler domain.


In [ ]:
import commpy
import numpy as np
import matplotlib.pyplot as plt

### OTFS Block Diagram

The entire process of OTFS can be summarized by the following block diagram :

<img src="Figures/OTFS_image.PNG" width="240*4" height="240*4" align="center"/>

<i> Orthogonal Time Frequency Space (OTFS) Modulation
and Applications,Tutorial at SPCOM 2020, IISc, Bangalore, July, 2020
Yi Hong, Emanuele Viterbo, Raviteja Patchava </i>

As shown in the above block diagram we can breakdown the OTFS scheme into the following steps :

1. Generation of symbols (QPSK,QAM etc.,) these are automatically assumed to be in the delay-doppler domain.
1. Conversion of these symbols into the time-frequency domain : ISFFT
1. Time-frequency domain to time domain i.e. transmission signal s(t) : Heisenberg Transform
1. Transmission over channel : Channel modelled in delay-doppler domain
1. Received signal converted into time-frequency domain using Wigner transform done through matched filtering.
1. Time frequency domain to delay doppler domain using SFFT.

### Building the system in the discrete domain

![Matrix](Figures/OTFS_matrix.PNG)
<i> Orthogonal Time Frequency Space (OTFS) Modulation
and Applications,Tutorial at SPCOM 2020, IISc, Bangalore, July, 2020
Yi Hong, Emanuele Viterbo, Raviteja Patchava </i>


In [ ]:
def gen_bits(size):
    return np.random.randint(2,size=(size,))

In [ ]:
M = 2048 # Number of points in the delay domain
N=  128  # Number of points in the doppler domain

total_symbols = M*N # Total Number of symbols that are required to be genrated for the given M and N
no_bits = 2*total_symbols # 4-QAM modulation is chosen i.e. 2 bits for every symbol

bits = gen_bits(no_bits) # Generating the information bits stream
QAM = commpy.QAMModem(4) # Creating the 4-QAM object
sym = QAM.modulate(bits) # Modulating the input bit stream into symbols

In [ ]:
sym_delay_dop = np.reshape(sym,(M,N))               # Constructing the symbol matrix
sym_delay_time = np.zeros((M,N), dtype = 'complex') # Initialising the delay time matrix 

for i in range(M) :                                 # Creating the delay timematrix by taking row wise fft            
    sym_delay_time[i,:] = np.fft.fft(sym_delay_dop[i,:])                    

In [ ]:
## Parallel to Serial Convertor
CP = 16  # Number of cyclic prefix added as a gaurd interval to prevent ISI 
x = np.zeros((M*N,), dtype = 'complex') # Creating the transmission array after P/S
x_CP = np.zeros((M*N + CP,), dtype = 'complex') # Adding CP 
for i in range(N):
    x[M*i:M*(i+1)] = sym_delay_time[:,i]
x_CP[:-CP] = x[::-1] # Reversing the x since we want the first symbol transmitted to be N=0 and not N=128
x_CP[-CP:] = x_CP[:CP] # Adding the cyclic prefix 

### Creating the Channel 

We now need to model the channel in the delay doppler domain. The following have to be decided to construct the channel : 

1. Number of taps : 5 taps
1. Channel model : Rayleigh
1. Power Delay Profile : Exponential i.e. IEEE 802.11


In [ ]:
def ieee802_11_model(sigma_tau,Ts):

    '''
    IEEE 802.11 channel model PDP generator
    Input:
       sigma_tau  : RMS delay spread
       Ts         : Sampling time
    Output:
       PDP        : Power delay profile

    MIMO-OFDM Wireless Communications with MATLAB¢ç   Yong Soo Cho, Jaekwon Kim, Won Young Yang and Chung G. Kang
    2010 John Wiley & Sons (Asia) Pte Ltd
    '''

    lmax = int(np.ceil(10*sigma_tau/Ts))
    sigma02=(1-np.exp(-Ts/sigma_tau))/(1-np.exp(-(lmax+1)*Ts/sigma_tau))
    l=np.array(range(lmax))
    PDP = sigma02*np.exp(-l*Ts/sigma_tau)

    return PDP

def OTFS_channel_gen(PDP=0, sigma_tau= 25*1e-9,Ts=50*1e-9): # I am not sure how to create the doppler array for channel 
                             #i am just taking what the SPCOM code is taking
    '''
    PDP = 0 : Uniform power delay profile
    PDP = 1 : IEEE 802.11 power delay profile
    
    sigma_tau : RMS Delay Spread
    Ts : Sampling period
    '''
    if(PDP == 1):
        PDP = ieee802_11_model(sigma_tau,Ts)
        taps = len(PDP)
        delay = np.array(range(taps))
        doppler = np.array(range(taps))
        channel_coeff = (np.random.rand(taps,))+ 1j*np.random.rand(taps,)*np.power(PDP,0.5)
    else :
        taps = 4
        delay = np.array(range(taps))
        doppler = np.array(range(taps))
        PDP = np.ones((1,taps))/taps
        channel_coeff = (np.random.rand(taps,))+ 1j*np.random.rand(taps,)*np.power(PDP,0.5)
    
    return delay,doppler,channel_coeff

In [ ]:
scale = 1e-9 
Ts = 50*scale         # Sampling time for our system
t_rms = 25*scale      # RMS delay spread

delay,doppler,h = OTFS_channel_gen(PDP=1,sigma_tau=t_rms,Ts=Ts) # Creating the channel

### Affect of the channel on the signal

![reveived_sig](Figures/received_signal.PNG)

 <i> Orthogonal Time Frequency Space (OTFS) Modulation
and Applications,Tutorial at SPCOM 2020, IISc, Bangalore, July, 2020
Yi Hong, Emanuele Viterbo, Raviteja Patchava </i>

In [ ]:
def noise_func(SNR,y):
    
    sig_pow = np.sum(y**2)
    noise_mag = np.power(np.power(10,(-1*SNR)/10) * sig_pow/2,0.5)
    noise = (np.random.rand(len(y),) + 1j*np.random.rand(len(y),)) * noise_mag
    
    return noise

In [ ]:

r_n = np.zeros(x_CP.size)  # Initialising the received signal
n= np.array(range(M*N+CP))
for i in range(len(delay)):
    r_n = r_n + h[i]*np.exp(1j*2*np.pi*doppler[i]*(n-delay[i]))*np.roll(x_CP,delay[i])

noise = noise_func(SNR=20,y=r_n)

y_n = r_n + noise

#### Receving end processing

1. Remove CP
1. Zak transform (time delay) -> (doppler,delay)

In [ ]:
## Constructing the time delay matrix
y_n_withoutCP = y_n[:-CP]
y_n_time_delay = np.reshape(y_n_withoutCP,(M,N))

In [ ]:
### Conversion of the time delay matrix to doppler delay matrix
y_delay_doppler = np.zeros((M,N),dtype='complex')
for i in range(M) :
    y_delay_doppler[i,:] = np.fft.ifft(y_n_time_delay[i,:])